# **PART 1**

In [1]:
%%shell
gdown --quiet 1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam
pip --quiet install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
COMPLAINTS_FN = 'complaints_sample.csv'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

In [ ]:
dt = sc.textFile(COMPLAINTS_FN, use_unicode=True).cache()
dt.take(2)

['Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID',
 '2015-12-31,Bank account or service,Checking account,"Making/receiving payments, sending money",,,,FIRSTBANK PUERTO RICO,PR,00902,Older American,N/A,Referral,2016-02-04,Closed with explanation,Yes,No,1723943']

In [ ]:
for x in dt.take(10):
    print(x)
    print('.')

Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
.
2015-12-31,Bank account or service,Checking account,"Making/receiving payments, sending money",,,,FIRSTBANK PUERTO RICO,PR,00902,Older American,N/A,Referral,2016-02-04,Closed with explanation,Yes,No,1723943
.
2016-03-15,Bank account or service,Other bank product/service,Problems caused by my funds being low,,,,FIRSTBANK PUERTO RICO,PR,00926,,Consent not provided,Web,2016-03-15,Closed with explanation,Yes,No,1833740
.
2016-10-24,Bank account or service,Checking account,"Account opening, closing, or management",,"In the month of XX/XX/2015, my email address ( XXXX ) was hacked and used to send messages to people associated with my business. At that time, transactions for the purchase and sales of products were made. The

In [ ]:
def combine(part_id,rows):
    if part_id == 0:
        next(rows)
    reserve = ''
    for row in rows:
        if row[:4].isnumeric()==True:
            if len(reserve)>0:
                yield reserve
            reserve = row
        else:
            reserve+=row

In [ ]:
import re
def modify(row):
    pattern = r'"[^"]*"'
    matches = re.finditer(pattern,row)
    positions = []
    for match in matches:
        positions.append((match.start(), match.end()))
    quot_modified = []
    if len(positions)>0:
        for x in positions:
            target_quot = row[x[0]:x[1]]
            replacement_quot = re.sub(",","_", target_quot) 
            quot_modified.append((replacement_quot,x))
        output_row = row
        for y in quot_modified:
            output_row = output_row[:y[1][0]]+y[0]+output_row[y[1][1]:]
        return output_row.lower()
    else:
        return row.lower()

In [ ]:
task1 = dt.mapPartitionsWithIndex(combine)\
        .map(modify)\
        .map(lambda x: x.split(','))\
        .map(lambda x: ((x[0][:4],x[1],x[7]),1))
task1.take(10)

[(('2015', 'bank account or service', 'firstbank puerto rico'), 1),
 (('2016', 'bank account or service', 'firstbank puerto rico'), 1),
 (('2016', 'bank account or service', 'wells fargo & company'), 1),
 (('2017', 'checking or savings account', 'comerica'), 1),
 (('2018', 'checking or savings account', 'wells fargo & company'), 1),
 (('2018',
   'checking or savings account',
   '"bank of america_ national association"'),
  1),
 (('2018', 'checking or savings account', 'hsbc north america holdings inc.'),
  1),
 (('2018', 'checking or savings account', 'jpmorgan chase & co.'), 1),
 (('2018', 'checking or savings account', 'navy federal credit union'), 1),
 (('2018', 'checking or savings account', 'jpmorgan chase & co.'), 1)]

In [ ]:
check = task1.filter(lambda x: x[0][0] in ['2018'])

In [ ]:
def remove_quot(row):
    if row[0][0].isalpha():
        return ','.join(row)
    else:
        out = [row[0][1:-1],row[1],row[2],row[3],row[4]]
        return ','.join(out)
outputTask1 = task1.groupByKey()\
        .mapValues(lambda x: sum(x))\
        .map(lambda x: ((x[0][0],x[0][1]),(x[1],1)))\
        .groupByKey()\
        .mapValues(lambda x: (sum([a[0] for a in x]),max([a[0] for a in x]),sum([a[1] for a in x])))\
        .map(lambda x: [x[0][1],x[0][0],str(x[1][0]),str(x[1][2]),str(round(100*x[1][1]/x[1][0],0))[:-2]])\
        .map(remove_quot)\
        .sortBy(lambda x: x[:])
outputTask1.take(20)

['bank account or service,2015,1,1,100',
 'bank account or service,2016,2,2,50',
 'checking or savings account,2017,1,1,100',
 'checking or savings account,2018,20,10,25',
 'checking or savings account,2019,461,72,13',
 'checking or savings account,2020,3,3,33',
 'consumer loan,2015,1,1,100',
 'consumer loan,2016,1,1,100',
 'consumer loan,2017,1,1,100',
 'credit card or prepaid card,2017,1,1,100',
 'credit card or prepaid card,2018,27,12,33',
 'credit card or prepaid card,2019,437,42,15',
 'credit card or prepaid card,2020,13,10,23',
 'credit card,2016,4,4,25',
 'credit card,2017,1,1,100',
 'credit reporting_ credit repair services_ or other personal consumer reports,2017,7,5,29',
 'credit reporting_ credit repair services_ or other personal consumer reports,2018,238,22,56',
 'credit reporting_ credit repair services_ or other personal consumer reports,2019,3113,203,50',
 'credit reporting_ credit repair services_ or other personal consumer reports,2020,144,10,51',
 'debt collection,20

In [ ]:
# delate folder
#!rm -rf 'report.csv'

# part 2

In [ ]:
!pip install google-cloud-dataproc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.5/307.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=dVHPqZDlY3OjaqlZhoygLdo4SZceEL&prompt=consent&access_type=offline&code_challenge=cMHT2OYMiGzblktM0zRoa4m98QwrT6BuojpoLro4yF8&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk4gS6gjABfhHOo_k2yyM2DBx5-Vic-JhXaAX95EourVw8IJRIzfmPCAwQd9qfFQ8w

You are now logged in as [zhiyuan.jin1201@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID

In [ ]:
!gcloud projects list

PROJECT_ID            NAME              PROJECT_NUMBER
bda-12345             bda-12345         369021837527
directed-will-384217  My First Project  913257743120


In [ ]:
!gcloud config set project bda-12345
!gcloud config set compute/region us-west1
!gcloud config set compute/zone us-west1-a
!gcloud config set dataproc/region us-west1

Updated property [core/project].
Updated property [compute/region].
Updated property [compute/zone].
Updated property [dataproc/region].


In [ ]:
!gcloud dataproc clusters create bdm-hw3 --enable-component-gateway --region us-west1 --zone us-west1-a --master-machine-type n1-standard-4 --master-boot-disk-size 500 --num-workers 2 --worker-machine-type n1-standard-4 --worker-boot-disk-size 500 --image-version 2.0-debian10 --project bda-12345

Waiting on operation [projects/bda-12345/regions/us-west1/operations/5a653e72-d97f-3d1b-a948-3a30d0e45cef].

Created [https://dataproc.googleapis.com/v1/projects/bda-12345/regions/us-west1/clusters/bdm-hw3] Cluster placed in zone [us-west1-a].


In [ ]:
!gcloud dataproc clusters list

NAME     PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE        SCHEDULED_DELETE
bdm-hw3  GCE       2                                       RUNNING  us-west1-a


In [ ]:
%%writefile BDM_HW3_23499013_Jin.py
#!/usr/bin/python
COMPLAINTS = 'gs://bdma/data/complaints.csv'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
dt = sc.textFile(COMPLAINTS, use_unicode=True).cache()

def combine(part_id,rows):
    if part_id == 0:
        next(rows)
    reserve = ''
    for row in rows:
        if row[:4].isnumeric()==True:
            if len(reserve)>0:
                yield reserve
            reserve = row
        else:
            reserve+=row

import re
def modify(row):
    pattern = r'"[^"]*"'
    matches = re.finditer(pattern,row)
    positions = []
    for match in matches:
        positions.append((match.start(), match.end()))
    # change all the commas-',' in the quotation mark to _, so that I can split with comma later on.
    # and use the modified sentence in the quotation mark to replace the original sentence in the quotation mark.
    quot_modified = []
    if len(positions)>0:
        for x in positions:
            target_quot = row[x[0]:x[1]]
            replacement_quot = re.sub(",","_", target_quot) 
            quot_modified.append((replacement_quot,x))
        output_row = row
        for y in quot_modified:
            output_row = output_row[:y[1][0]]+y[0]+output_row[y[1][1]:]
        return output_row.lower()
    else:
        return row.lower()

task1 = dt.mapPartitionsWithIndex(combine)\
        .map(modify)\
        .map(lambda x: x.split(','))\
        .map(lambda x: ((x[0][:4],x[1],x[7]),1))

def remove_quot(row):
    if row==none:
      pass
    elif row[0][0].isalpha():
        return ','.join(row)
    else:
        out = [row[0][1:-1],row[1],row[2],row[3],row[4]]
        return ','.join(out)
        
outputTask1 = task1.groupByKey()\
        .mapValues(lambda x: sum(x))\
        .map(lambda x: ((x[0][0],x[0][1]),(x[1],1)))\
        .groupByKey()\
        .mapValues(lambda x: (sum([a[0] for a in x]),max([a[0] for a in x]),sum([a[1] for a in x])))\
        .map(lambda x: [x[0][1],x[0][0],str(x[1][0]),str(x[1][2]),str(round(100*x[1][1]/x[1][0],0))[:-2]])\
        .map(remove_quot)\
        .sortBy(lambda x: x)
outputTask1.saveAsTextFile('report.csv')

Overwriting BDM_HW3_23499013_Jin.py


In [ ]:
!gcloud dataproc jobs submit pyspark --cluster bdm-hw3 BDM_HW3_23499013_Jin.py -- gs://bdma/data/complaints.csv gs://bdma/shared/2023_spring/HW3/23499013_Jin

Job [de33e22e86ca4ba5bdefd07d1a57df5e] submitted.
Waiting for job output...
23/04/20 02:14:55 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/04/20 02:14:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/04/20 02:14:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/04/20 02:14:55 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
23/04/20 02:14:55 INFO org.sparkproject.jetty.util.log: Logging initialized @3298ms to org.sparkproject.jetty.util.log.Slf4jLog
23/04/20 02:14:55 INFO org.sparkproject.jetty.server.Server: jetty-9.4.40.v20210413; built: 2021-04-13T20:42:42.668Z; git: b881a572662e1943a14ae12e7e1207989f218b74; jvm 1.8.0_362-b09
23/04/20 02:14:55 INFO org.sparkproject.jetty.server.Server: Started @3417ms
23/04/20 02:14:55 INFO org.sparkproject.jetty.server.AbstractConnector: Started ServerConnector@32bcf606{HTTP/1.1, (http/1.1)}{0.0.0.0:34853}
23/04/20 02:14:56 INFO org.apache.hadoop.yarn.client.RMPro

In [ ]:
!gsutil ls gs://bdma/shared/2023_spring/HW3/

gs://bdma/shared/2023_spring/HW3/
gs://bdma/shared/2023_spring/HW3/14211712_Salas/
gs://bdma/shared/2023_spring/HW3/15304719_Graig/
gs://bdma/shared/2023_spring/HW3/23513476_Gabr/
gs://bdma/shared/2023_spring/HW3/23621671_Kannam/
gs://bdma/shared/2023_spring/HW3/23668189_Matuszewski/
gs://bdma/shared/2023_spring/HW3/23703839_Feng/
gs://bdma/shared/2023_spring/HW3/23735863_Ghimire/
gs://bdma/shared/2023_spring/HW3/24363838_Lau/
gs://bdma/shared/2023_spring/HW3/24369480_Chandani/
gs://bdma/shared/2023_spring/HW3/24369705_Khandale/
gs://bdma/shared/2023_spring/HW3/24369977_Gulve/
gs://bdma/shared/2023_spring/HW3/24373710_Uddin/
gs://bdma/shared/2023_spring/HW3/24379388_Quadri/
gs://bdma/shared/2023_spring/HW3/24379541_Desai/
gs://bdma/shared/2023_spring/HW3/24379542_doroudian/
gs://bdma/shared/2023_spring/HW3/24438996_Radaelli/
